In [3]:
import scipy as sp
import numpy as np
import pandas as pd
import timeit
import re
import json
import pickle
import fastparquet
import os
os.chdir('/mnt/t48/bighomes-active/sfeng/patentdiffusion/')
seed = 3
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
import datetime
import string

In [26]:
# ns = fastparquet.ParquetFile("DataStore/2018-11/naics_name_sim_claims_1120.parq").to_pandas()
# pdf = fastparquet.ParquetFile("RawData/Cleaned/patent_loc_unique_us_0628.parq").to_pandas(["patent", "gyear"])
# ns["op_gyear"] = ns["op"].map(dict(zip(pdf["patent"], pdf["gyear"])))
# ns["year_match"] = (ns["tp_gyear"] == ns["op_gyear"])

for c in ["num_shared_abs_stemmed", "num_shared_claims_stemmed"]:
    ns["norm_"+c] = np.nan
    ns.loc[ns[c].notnull(), "norm_"+c] = scaler.fit_transform(ns.loc[ns[c].notnull(), c].values.reshape(-1, 1))

In [32]:
ns.columns

Index(['tp', 'op', 'sim_claims_docvecs', 'norm_sim_claims_docvecs',
       'sim_docvecs', 'sim_ldavecs', 'tp_gyear', 'tp_naics_name',
       'op_naics_name', 'op_primclass', 'op_inv_msa', 'inv_msa_match',
       'primclass_match', 'norm_sim_ldavecs', 'norm_sim_docvecs', 'year_group',
       'common_est_inv', 'common_pat_inv', 'lawyer_match', 'num_common_cited',
       'norm_num_common_cited', 'tp_pct_common_cited',
       'norm_tp_pct_common_cited', 'common_cited_match', 'common_npc_match',
       'mean_sim_docvecs_pc', 'mean_sim_docvecs_pc_msa',
       'norm_mean_sim_docvecs_pc', 'norm_mean_sim_docvecs_pc_msa',
       'num_common_npc', 'norm_num_common_npc', 'tp_primclass_FE',
       'tp_inv_msa_FE', 'tp_examiner_FE', 'tp_lawyer_FE', 'examiner_match',
       'num_shared_abs_stemmed', 'num_shared_claims_stemmed', 'tp_primclass',
       'op_gyear', 'year_match', 'norm_num_shared_abs_stemmed',
       'norm_num_shared_claims_stemmed'],
      dtype='object')

In [37]:
cols = ["inv_msa_match", "primclass_match", "common_pat_inv", "common_cited_match", "common_npc_match", "year_match",
        "lawyer_match", "examiner_match",]
mes = ["norm_num_shared_claims_stemmed", "norm_num_shared_abs_stemmed", "norm_sim_docvecs", "norm_sim_claims_docvecs"]
for m in mes:
    print(m)
    res = pd.DataFrame()
    for c in cols:
        r = ns[["year_group", m, c]].groupby(["year_group", c]).mean().rename(columns={m:c})
        res = pd.concat([res,r], axis=1)

    display(np.round(res,3))

norm_num_shared_claims_stemmed


inv_msa_match  primclass_match  common_pat_inv  \
year_group inv_msa_match                                                   
1975-85    False                  0.365            0.303           0.379   
           True                   0.439            1.010           5.027   
1985-95    False                  0.177            0.116           0.197   
           True                   0.282            0.766           4.507   
1995-05    False                 -0.097           -0.165          -0.075   
           True                   0.012            0.476           3.899   
2005-15    False                 -0.194           -0.248          -0.176   
           True                  -0.111            0.255           2.956   

                          common_cited_match  common_npc_match  year_match  \
year_group inv_msa_match                                                     
1975-85    False                       0.380             0.382       0.381   
           True                        4.392            17.268       0.387   
1985-95    False                       0.197             0.202       0.204   
           True                        3.753            10.887       0.195   
1995-05    False                      -0.075            -0.071      -0.072   
           True                        2.232             6.126      -0.064   
2005-15    False                      -0.178            -0.174      -0.174   
           True                        1.257             3.475      -0.170   

                          lawyer_match  examiner_match  
year_group inv_msa_match                                
1975-85    False                 0.377           0.367  
           True                  1.516           1.301  
1985-95    False                 0.195           0.189  
           True                  1.426           1.088  
1995-05    False                -0.074          -0.076  
           True                  1.044           0.754  
2005-15    False                -0.175          -0.176  
           True                  0.579           0.579

norm_num_shared_abs_stemmed


inv_msa_match  primclass_match  common_pat_inv  \
year_group inv_msa_match                                                   
1975-85    False                 -0.071           -0.099          -0.056   
           True                   0.009            0.311           4.105   
1985-95    False                 -0.054           -0.088          -0.041   
           True                   0.022            0.307           3.985   
1995-05    False                  0.025           -0.006           0.046   
           True                   0.131            0.381           3.877   
2005-15    False                 -0.024           -0.061          -0.007   
           True                   0.056            0.327           3.245   

                          common_cited_match  common_npc_match  year_match  \
year_group inv_msa_match                                                     
1975-85    False                      -0.055            -0.052      -0.052   
           True                        3.818            17.144      -0.053   
1985-95    False                      -0.041            -0.036      -0.036   
           True                        3.179            10.758      -0.038   
1995-05    False                       0.046             0.050       0.050   
           True                        2.374             6.958       0.054   
2005-15    False                      -0.008            -0.004      -0.005   
           True                        1.262             3.826       0.002   

                          lawyer_match  examiner_match  
year_group inv_msa_match                                
1975-85    False                -0.058          -0.065  
           True                  1.190           0.665  
1985-95    False                -0.043          -0.046  
           True                  1.164           0.645  
1995-05    False                 0.047           0.046  
           True                  1.238           0.742  
2005-15    False                -0.006          -0.007  
           True                  0.793           0.679

norm_sim_docvecs


inv_msa_match  primclass_match  common_pat_inv  \
year_group inv_msa_match                                                   
1975-85    False                 -0.046           -0.103          -0.043   
           True                  -0.027            0.439           1.332   
1985-95    False                 -0.078           -0.138          -0.067   
           True                  -0.026            0.414           1.510   
1995-05    False                 -0.038           -0.111          -0.025   
           True                   0.020            0.482           1.422   
2005-15    False                  0.050           -0.013           0.063   
           True                   0.107            0.514           1.379   

                          common_cited_match  common_npc_match  year_match  \
year_group inv_msa_match                                                     
1975-85    False                      -0.043            -0.042      -0.042   
           True                        1.565               NaN      -0.042   
1985-95    False                      -0.067            -0.065      -0.066   
           True                        1.305             1.869      -0.063   
1995-05    False                      -0.026            -0.024      -0.024   
           True                        1.159             1.742      -0.023   
2005-15    False                       0.062             0.064       0.063   
           True                        0.940             1.811       0.070   

                          lawyer_match  examiner_match  
year_group inv_msa_match                                
1975-85    False                -0.043          -0.051  
           True                  0.187           0.527  
1985-95    False                -0.067          -0.074  
           True                  0.298           0.526  
1995-05    False                -0.025          -0.028  
           True                  0.455           0.501  
2005-15    False                 0.064           0.062  
           True                  0.446           0.575

norm_sim_claims_docvecs


inv_msa_match  primclass_match  common_pat_inv  \
year_group inv_msa_match                                                   
1975-85    False                 -0.012           -0.069          -0.004   
           True                   0.028            0.520           1.960   
1985-95    False                 -0.054           -0.112          -0.043   
           True                   0.005            0.433           1.990   
1995-05    False                 -0.021           -0.086          -0.007   
           True                   0.046            0.464           1.839   
2005-15    False                  0.009           -0.059           0.022   
           True                   0.069            0.499           1.653   

                          common_cited_match  common_npc_match  year_match  \
year_group inv_msa_match                                                     
1975-85    False                      -0.004            -0.002      -0.002   
           True                        1.843             5.320      -0.002   
1985-95    False                      -0.043            -0.040      -0.040   
           True                        1.726             4.308      -0.042   
1995-05    False                      -0.007            -0.005      -0.005   
           True                        1.377             2.835      -0.000   
2005-15    False                       0.021             0.023       0.023   
           True                        0.967             2.171       0.026   

                          lawyer_match  examiner_match  
year_group inv_msa_match                                
1975-85    False                -0.004          -0.014  
           True                  0.353           0.661  
1985-95    False                -0.043          -0.049  
           True                  0.456           0.567  
1995-05    False                -0.006          -0.009  
           True                  0.544           0.569  
2005-15    False                 0.022           0.021  
           True                  0.472           0.619

In [65]:
0.028/1.96

0.014285714285714287

In [38]:
cols = ["inv_msa_match", "primclass_match", "common_pat_inv", "common_cited_match", "common_npc_match", "year_match",
        "lawyer_match", "examiner_match",]
mes = ["num_shared_claims_stemmed", "num_shared_abs_stemmed", "sim_docvecs", "sim_claims_docvecs"]
for m in mes:
    print(m)
    res = pd.DataFrame()
    for c in cols:
        r = ns[["year_group", m, c]].groupby(["year_group", c]).mean().rename(columns={m:c})
        res = pd.concat([res,r], axis=1)

    display(np.round(res,3))

num_shared_claims_stemmed


inv_msa_match  primclass_match  common_pat_inv  \
year_group inv_msa_match                                                   
1975-85    False                  2.521            2.391           2.550   
           True                   2.676            3.862          12.214   
1985-95    False                  2.130            2.004           2.171   
           True                   2.349            3.356          11.134   
1995-05    False                  1.561            1.419           1.606   
           True                   1.786            2.753           9.870   
2005-15    False                  1.359            1.248           1.396   
           True                   1.532            2.291           7.908   

                          common_cited_match  common_npc_match  year_match  \
year_group inv_msa_match                                                     
1975-85    False                       2.551             2.557       2.555   
           True                       10.894            37.667       2.567   
1985-95    False                       2.171             2.182       2.186   
           True                        9.566            24.400       2.167   
1995-05    False                       1.607             1.615       1.613   
           True                        6.404            14.500       1.630   
2005-15    False                       1.392             1.401       1.400   
           True                        4.377             8.988       1.410   

                          lawyer_match  examiner_match  
year_group inv_msa_match                                
1975-85    False                 2.547           2.525  
           True                  4.914           4.467  
1985-95    False                 2.168           2.154  
           True                  4.728           4.025  
1995-05    False                 1.608           1.603  
           True                  3.934           3.329  
2005-15    False                 1.398           1.396  
           True                  2.966           2.966

num_shared_abs_stemmed


inv_msa_match  primclass_match  common_pat_inv  \
year_group inv_msa_match                                                   
1975-85    False                  2.618            2.554           2.653   
           True                   2.802            3.499          12.251   
1985-95    False                  2.657            2.579           2.687   
           True                   2.833            3.490          11.976   
1995-05    False                  2.839            2.767           2.888   
           True                   3.085            3.660          11.727   
2005-15    False                  2.727            2.641           2.766   
           True                   2.911            3.535          10.268   

                          common_cited_match  common_npc_match  year_match  \
year_group inv_msa_match                                                     
1975-85    False                       2.654             2.661       2.662   
           True                       11.589            42.333       2.660   
1985-95    False                       2.688             2.698       2.700   
           True                       10.115            27.600       2.694   
1995-05    False                       2.889             2.898       2.898   
           True                        8.259            18.833       2.905   
2005-15    False                       2.764             2.772       2.770   
           True                        5.692            11.608       2.787   

                          lawyer_match  examiner_match  
year_group inv_msa_match                                
1975-85    False                 2.648           2.633  
           True                  5.528           4.316  
1985-95    False                 2.683           2.675  
           True                  5.466           4.270  
1995-05    False                 2.890           2.887  
           True                  5.639           4.493  
2005-15    False                 2.768           2.766  
           True                  4.612           4.348

sim_docvecs


inv_msa_match  primclass_match  common_pat_inv  \
year_group inv_msa_match                                                   
1975-85    False                  0.124            0.117           0.125   
           True                   0.127            0.190           0.310   
1985-95    False                  0.120            0.112           0.121   
           True                   0.127            0.186           0.334   
1995-05    False                  0.125            0.116           0.127   
           True                   0.133            0.196           0.323   
2005-15    False                  0.137            0.129           0.139   
           True                   0.145            0.200           0.317   

                          common_cited_match  common_npc_match  year_match  \
year_group inv_msa_match                                                     
1975-85    False                       0.125             0.125       0.125   
           True                        0.342               NaN       0.125   
1985-95    False                       0.121             0.122       0.122   
           True                        0.307             0.383       0.122   
1995-05    False                       0.127             0.127       0.127   
           True                        0.287             0.366       0.127   
2005-15    False                       0.139             0.139       0.139   
           True                        0.257             0.375       0.140   

                          lawyer_match  examiner_match  
year_group inv_msa_match                                
1975-85    False                 0.125           0.124  
           True                  0.156           0.202  
1985-95    False                 0.121           0.121  
           True                  0.171           0.202  
1995-05    False                 0.127           0.127  
           True                  0.192           0.198  
2005-15    False                 0.139           0.139  
           True                  0.191           0.208

sim_claims_docvecs


inv_msa_match  primclass_match  common_pat_inv  \
year_group inv_msa_match                                                   
1975-85    False                  0.095            0.088           0.096   
           True                   0.101            0.168           0.364   
1985-95    False                  0.089            0.082           0.091   
           True                   0.098            0.156           0.368   
1995-05    False                  0.094            0.085           0.096   
           True                   0.103            0.160           0.348   
2005-15    False                  0.098            0.089           0.100   
           True                   0.106            0.165           0.322   

                          common_cited_match  common_npc_match  year_match  \
year_group inv_msa_match                                                     
1975-85    False                       0.096             0.097       0.097   
           True                        0.348             0.823       0.097   
1985-95    False                       0.091             0.091       0.091   
           True                        0.332             0.685       0.091   
1995-05    False                       0.096             0.096       0.096   
           True                        0.285             0.484       0.097   
2005-15    False                       0.100             0.100       0.100   
           True                        0.229             0.393       0.101   

                          lawyer_match  examiner_match  
year_group inv_msa_match                                
1975-85    False                 0.096           0.095  
           True                  0.145           0.187  
1985-95    False                 0.091           0.090  
           True                  0.159           0.174  
1995-05    False                 0.096           0.096  
           True                  0.171           0.175  
2005-15    False                 0.100           0.100  
           True                  0.161           0.181

In [33]:
ps = fastparquet.ParquetFile("DataStore/2018-11/primclass_sim_claims_1120.parq").to_pandas()

In [42]:
cols = ["inv_msa_match", "common_pat_inv", "common_cited_match", "common_npc_match",
        "lawyer_match", "examiner_match",]
mes = ["num_shared_claims_stemmed", "num_shared_abs_stemmed", "sim_docvecs", "sim_claims_docvecs"]
for m in mes:
    print(m)
    res = pd.DataFrame()
    for c in cols:
        r = ps[["year_group", m, c]].groupby(["year_group", c]).mean().rename(columns={m:c})
        res = pd.concat([res,r], axis=1)

    display(np.round(res,3))

num_shared_claims_stemmed


inv_msa_match  common_pat_inv  common_cited_match  \
year_group inv_msa_match                                                      
1975-85    False                  4.081           4.102               4.095   
           True                   4.366          13.640               9.503   
1985-95    False                  3.536           3.566               3.555   
           True                   3.870          12.684               8.077   
1995-05    False                  2.728           2.765               2.760   
           True                   3.026          10.195               5.647   
2005-15    False                  2.312           2.341               2.323   
           True                   2.510           8.562               4.469   

                          common_npc_match  lawyer_match  examiner_match  
year_group inv_msa_match                                                  
1975-85    False                     4.132         4.104           4.044  
           True                     25.800         8.686           4.623  
1985-95    False                     3.598         3.569           3.530  
           True                     16.852         7.663           4.087  
1995-05    False                     2.789         2.774           2.760  
           True                     15.202         6.055           3.239  
2005-15    False                     2.354         2.346           2.340  
           True                     10.972         5.004           2.738

num_shared_abs_stemmed


inv_msa_match  common_pat_inv  common_cited_match  \
year_group inv_msa_match                                                      
1975-85    False                  3.811           3.831               3.825   
           True                   4.106          14.220               9.461   
1985-95    False                  3.779           3.809               3.796   
           True                   4.133          14.030               8.930   
1995-05    False                  3.931           3.971               3.964   
           True                   4.268          12.708               7.638   
2005-15    False                  3.589           3.629               3.611   
           True                   3.843          10.654               5.871   

                          common_npc_match  lawyer_match  examiner_match  
year_group inv_msa_match                                                  
1975-85    False                     3.864         3.832           3.758  
           True                     31.600         8.954           4.458  
1985-95    False                     3.844         3.812           3.769  
           True                     29.407         8.384           4.389  
1995-05    False                     4.000         3.982           3.969  
           True                     19.179         7.910           4.494  
2005-15    False                     3.643         3.634           3.631  
           True                     14.738         6.577           4.017

sim_docvecs


inv_msa_match  common_pat_inv  common_cited_match  \
year_group inv_msa_match                                                      
1975-85    False                  0.190           0.191               0.190   
           True                   0.197           0.386               0.322   
1985-95    False                  0.182           0.184               0.183   
           True                   0.193           0.364               0.304   
1995-05    False                  0.184           0.185               0.185   
           True                   0.195           0.359               0.287   
2005-15    False                  0.188           0.190               0.189   
           True                   0.197           0.346               0.272   

                          common_npc_match  lawyer_match  examiner_match  
year_group inv_msa_match                                                  
1975-85    False                     0.191         0.191           0.188  
           True                      0.355         0.274           0.208  
1985-95    False                     0.184         0.184           0.182  
           True                      0.503         0.263           0.199  
1995-05    False                     0.186         0.186           0.185  
           True                      0.394         0.264           0.198  
2005-15    False                     0.190         0.190           0.190  
           True                      0.418         0.261           0.205

sim_claims_docvecs


inv_msa_match  common_pat_inv  common_cited_match  \
year_group inv_msa_match                                                      
1975-85    False                  0.169           0.170               0.170   
           True                   0.178           0.398               0.337   
1985-95    False                  0.158           0.159               0.158   
           True                   0.168           0.392               0.307   
1995-05    False                  0.153           0.154               0.154   
           True                   0.164           0.374               0.269   
2005-15    False                  0.151           0.152               0.151   
           True                   0.159           0.348               0.246   

                          common_npc_match  lawyer_match  examiner_match  
year_group inv_msa_match                                                  
1975-85    False                     0.171         0.170           0.167  
           True                      0.557         0.276           0.190  
1985-95    False                     0.160         0.159           0.157  
           True                      0.549         0.261           0.179  
1995-05    False                     0.155         0.155           0.154  
           True                      0.460         0.252           0.172  
2005-15    False                     0.153         0.152           0.152  
           True                      0.435         0.244           0.172

In [63]:
cols = ["common_cited_match", "common_npc_match",]
mes = ["num_shared_claims_stemmed", "num_shared_abs_stemmed", "sim_docvecs", "sim_claims_docvecs"]
m = "sim_docvecs"
# Groupby common cited
cc_g = ps.loc[ps["common_cited_match"] == True, [m, "year_group"]].groupby(["year_group"])
cn_g = ps.loc[ps["common_npc_match"] == True, [m, "year_group"]].groupby(["year_group"])
p_g = ps[[m, "year_group"]].groupby(["year_group"])

# Mean
cc_m = cc_g.mean().reset_index().rename(columns={"year_group": "Year Group", m: "Common Pat Cite"})
cn_m = cn_g.mean().reset_index().rename(columns={m: "Common Non-Pat Cite"})
p_m = p_g.mean().reset_index().rename(columns={m: "Primclass Match"})
cc_m["Common Non-Pat Cite"] = cn_m["Common Non-Pat Cite"]
cc_m["Primclass Match"] = p_m["Primclass Match"]
# p-value
p = [sp.stats.ttest_ind(g[m], cn_g.get_group(n)[m], equal_var=False, nan_policy="omit")[1] for n,g in cc_g]
cc_m["$p$-value"] = p
cc_m = np.round(cc_m.set_index("Year Group").T,3)

display(cc_m)
print(cc_m.to_latex(escape=False, column_format="lcccc"))

Year Group,1975-85,1985-95,1995-05,2005-15
Common Pat Cite,0.322,0.304,0.287,0.272
Common Non-Pat Cite,0.355,0.503,0.394,0.418
Primclass Match,0.191,0.184,0.186,0.190
$p$-value,0.761,0.000,0.000,0.000


\begin{tabular}{lcccc}
\toprule
Year Group &  1975-85 &  1985-95 &  1995-05 &  2005-15 \\
\midrule
Common Pat Cite     &    0.322 &    0.304 &    0.287 &    0.272 \\
Common Non-Pat Cite &    0.355 &    0.503 &    0.394 &    0.418 \\
Primclass Match     &    0.191 &    0.184 &    0.186 &    0.190 \\
\$p\$-value           &    0.761 &    0.000 &    0.000 &    0.000 \\
\bottomrule
\end{tabular}



In [60]:
cols = ["common_cited_match", "common_npc_match",]
mes = ["num_shared_claims_stemmed", "num_shared_abs_stemmed", "sim_docvecs", "sim_claims_docvecs"]
m = "sim_docvecs"
# Groupby common cited
cc_g = ns.loc[ns["common_cited_match"] == True, [m, "year_group"]].groupby(["year_group"])
cn_g = ns.loc[ns["common_npc_match"] == True, [m, "year_group"]].groupby(["year_group"])

# Mean
cc_m = cc_g.mean().reset_index().rename(columns={"year_group": "Year Group", m: "Common Pat Cite"})
cn_m = cn_g.mean().reset_index().rename(columns={m: "Common Non-Pat Cite"})
cc_m["Common Non-Pat Cite"] = cn_m["Common Non-Pat Cite"]
# p-value
p = [sp.stats.ttest_ind(g[m], cn_g.get_group(n)[m], equal_var=False, nan_policy="omit")[1] for n,g in cc_g]
cc_m["$p$-value"] = p
cc_m = np.round(cc_m.set_index("Year Group").T,3)

display(cc_m)
print(cc_m.to_latex(escape=False, column_format="lcccc"))

Year Group,1975-85,1985-95,1995-05,2005-15
Common Pat Cite,0.341869,0.306677,0.287022,0.257394
Common Non-Pat Cite,NaN,0.382807,0.365765,0.375029
$p$-value,--,--,0.0183376,2.38385e-07


\begin{tabular}{lcccc}
\toprule
Year Group &   1975-85 &   1985-95 &    1995-05 &      2005-15 \\
\midrule
Common Pat Cite     &  0.341869 &  0.306677 &   0.287022 &     0.257394 \\
Common Non-Pat Cite &       NaN &  0.382807 &   0.365765 &     0.375029 \\
\$p\$-value           &        -- &        -- &  0.0183376 &  2.38385e-07 \\
\bottomrule
\end{tabular}



In [52]:
cc_g.mean()

,sim_docvecs
year_group,
1975-85,0.322329
1985-95,0.303858
1995-05,0.286889
2005-15,0.271999
